In [1]:
!pip install numpy
!pip install matplotlib
!pip install tqdm

In [20]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import matplotlib.animation as anim
import time
from mpl_toolkits.mplot3d import Axes3D

# a) ¿Cuál es el significado físico de K?

K en este caso es una magnitud de fuerza entre esferas. 

# b) ¿Es conservativa la fuerza?

En este caso la fuerza si es conservativa.

In [21]:
class Particle():
    
        # init
        def __init__(self, r0,v0,a0,t,m,radius,Id):

            self.dt  = t[1] - t[0]

            self.r = r0
            self.v = v0
            self.a = a0

            self.rVector = np.zeros( (len(t),len(r0)) )
            self.vVector = np.zeros( (len(t),len(v0)) )
            self.aVector = np.zeros( (len(t),len(a0)) )

            self.m = m
            self.radius = radius
            self.Id = Id

            self.MomentumVector = np.zeros((len(t),len(v0)))
            self.kVector = np.zeros((len(t),1))
            self.pVector = np.zeros((len(t),1))

            self.Ep = 0
            self.Force = self.m * self.a

            #Tenemos que K es 100
            self.K = 100
        
    # Method
        def Evolution(self,i):

            self.SetPosition(i,self.r)
            self.SetVelocity(i,self.v)
            self.SetMomentum(i, self.m*self.v)
            self.SetEp(i, self.GetPotentialEnergy())
            self.SetEk(i, self.GetKineticEnergy())

            self.a = self.Force/self.m

            # Euler method
            self.r += self.dt * self.v
            self.v += self.dt * self.a
        
        def CheckWallLimits(self, limits, dim =2):
            for i in range(dim):
                if self.r[i] + self.radius > limits[i] and self.v[i] > 0:
                    self.v[i] = -self.v[i]
                if self.r[i] - self.radius < -limits[i] and self.v[i] < 0:
                    self.v[i] = -self.v[i]

        # Setters

        def ResetForce(self):
            self.Force[:] = 0
            self.a[:] = 0
            self.Ep = 0
            self.Niter = 0

        def SetPosition(self,i,r):
            self.rVector[i] = r

        def SetVelocity(self,i,v):
            self.vVector[i] = v

        def SetMomentum(self, i, p):
            self.MomentumVector[i] = p

        def SetEk(self, i, Ek):
            self.kVector[i] = Ek

        def SetEp(self, i, Ep):
            self.pVector[i] = Ep

        # Getters  
        #Creamos la Fuerza
        def GetForce(self, p):
            #¿Qué distancia hay entre partículas?
            #self.r es la posición de la partícula a la quele calculamos la fuerza menos la posición de la otra.
            d = np.linalg.norm(self.r - p.GetPosition())
            compression = self.radius + p.GetR() - d 
            
            if compression > 0:
                Fn = self.K * compression**3/d
                self.Force = np.add(self.Force, Fn*(self.r - p.GetPosition()))
                self.Ep += self.K * compression**4 / 4
                
        def GetPosition(self):
            return self.r
    
        def GetPositionVector(self):
            return self.rVector
                
        def GetRPositionVector(self):
            return self.RrVector 
    
        def GetVelocityVector(self):
            return self.vVector
    
        def GetReduceVelocity(self):
            return self.RvVector
    
        def GetMomentumVector(self):
            return self.MomentumVector
    
        def GetR(self):
            return self.radius
    
        def GetKineticEnergy(self): 
            return 0.5*self.m*np.linalg.norm(self.v)**2

        def GetPotentialEnergy(self):
            return 0.5*self.Ep/2
        
        def GetNetForce(self):
            return self.Force

        def ReduceSize(self,factor):

            self.RrVector = np.array([self.rVector[0]]) # initial condition

            for i in range(1,len(self.rVector)):
                if i%factor == 0:
                    self.RrVector = np.vstack([self.RrVector,self.rVector[i]])
                    
            self.RvVector = np.array([self.vVector[0]])
            for i in range(1,len(self.vVector)):
                if i%factor == 0:
                    self.RvVector = np.vstack([self.RvVector,self.rVector[i]])

In [22]:
def GetParticles(NParticles,Limit,Velo,Dim=2,dt=0.1):
    
    Particles_ = []
    
    for i in range(NParticles):
        
        x0 = np.random.uniform(-Limit+1.0, Limit-1.0, size=Dim)
        v0 = np.random.uniform(-Velo, Velo, size=Dim)
        a0 = np.zeros(Dim)
        
        p = Particle(x0,v0,a0,t,1.,2.0,i)
        
        Particles_.append(p)
        
    return Particles_

In [23]:
A = np.array([20.,20.])

In [24]:
def RunSimulation(t,NParticles = 10, Velo = 6):
    
    Particles = GetParticles(NParticles,A[1],Velo = Velo,dt=dt)
    
    for it in tqdm(range(len(t))): # Evolucion temporal
        
        NetForce = np.array([0,0,0])
        
        for i in range(len(Particles)):
            for j in range(len(Particles)):
                if i != j:
                    Particles[i].GetForce(Particles[j])
                    
        for i in range(len(Particles)):
            Particles[i].Evolution(it)
            Particles[i].ResetForce()
            Particles[i].CheckWallLimits(A)
        
    return Particles

In [25]:
dt = 0.0001 
tmax = 1
t = np.arange(0, tmax, dt)
Particles = RunSimulation(t, 10, Velo = 10)

100%|██████████| 10000/10000 [00:11<00:00, 858.07it/s]


In [26]:
Kener = Particles[0].kVector
EnerP = Particles[0].pVector
EnerT = Particles[0].kVector + Particles[0].pVector

for i in range(1, len(Particles)):
    Kener = np.add(Kener, Particles[i].kVector)
    EnerP = np.add(EnerP, Particles[i].pVector)
    EnerT = np.add(EnerT, Particles[i].kVector + Particles[i].pVector)
    
Momentum = Particles[0].GetMomentumVector()

for i in range(1, len(Particles)):
    Momentum = np.add(Momentum, Particles[i].GetMomentumVector())

In [37]:
def ReduceTime(t,factor):
    for p in Particles:
        p.ReduceSize(factor)
        
    Newt = []
    for i in range(len(t)):
        if i%factor == 0:
            Newt.append(t[i])
    return np.array(Newt)

In [38]:
redt = ReduceTime(t,100)

In [52]:
def create_particles():
    r= 2.
    m= 1.
    r1= np.array([3,0])
    v1= np.array([20,0])
    a1= np.array([0,0])
    p1= Particle(r1,v1,a1,t,m,r,0)
    r2= np.array([-2,2])
    v2= np.array([0,0])
    p2= Particle(r2,v2,a1,t,m,r,1)
    r3= np.array([-15,-15])
    p3= Particle(r3,v2,a1,t,m,r,2)
    return [p1,p2,p3]

Particles= create_particles()

In [29]:
dt = 0.0001 
tmax = 1
t = np.arange(0, tmax, dt)
Particles = RunSimulation(t, 10, Velo = 10)

100%|██████████| 10000/10000 [00:11<00:00, 842.87it/s]


In [45]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1)


def init():
    ax.set_xlabel(r'$x[m]$', fontsize = 15)
    ax.set_ylabel(r'$y[m]$', fontsize = 15)
    ax.set_xlim(-Limits[0],Limits[0])
    ax.set_ylim(-Limits[1],Limits[1])

Colors = ['y', 'r', 'k']

def Update(i):
    
    plot = ax.clear()
    init()
    plot = ax.set_title(r'$t=%.2f \ seconds$' %(redt[i]), fontsize=15)
    
    j = 0
    for p in Particles:
        x = p.GetRPositionVector()[i,0]
        y = p.GetRPositionVector()[i,1] 
        
        vx = p.GetReduceVelocity()[i,0]
        vy = p.GetReduceVelocity()[i,1] 
        
        
        circle = plt.Circle( (x,y), p.GetR(), fill=True, color = 'k', label = str(j))
        plot = ax.add_patch(circle)
        plot = ax.arrow(x, y, vx, vy, color = 'r', head_width = 1.0)
        j += 1
        
    plot = ax.legend(loc=1)

    return plot


Animation = anim.FuncAnimation(fig,Update,frames=len(redt),init_func=init)

<IPython.core.display.Javascript object>

In [46]:
Writer = anim.writers['ffmpeg']
writer_ = Writer(fps=50, metadata=dict(artist='Me'))
Animation.save('Billar.mp4', writer=writer_)

RuntimeError: Requested MovieWriter (ffmpeg) not available

In [48]:
Kener= Particles[0].kVector
EnerP= Particles[0].pVector
EnerT= Particles[0].kVector + Particles[0].pVector
for i in range(1,len(Particles)):
    Kener=np.add(Kener,Particles[i].kVector )
    EnerP=np.add(EnerP,Particles[i].pVector )
    EnerT=np.add(EnerT,Particles[i].kVector + Particles[i].pVector )  
Momentum=Particles[0].GetMomentumVector()

for i in range(1,len(Particles)):
    Momentum=np.add(Momentum, Particles[i].GetMomentumVector())
    
fig1=plt.figure(figsize=(10,5))
ax1=fig1.add_subplot(1,1,1)
ax1.set_xlabel(r"$t[s]$")
ax1.set_ylabel(r"$E[J]$")
ax1.plot(t,Kener,"--",label="Energía cinética")
ax1.plot(t,EnerP,"--",label="Energía potencial")
ax1.plot(t,EnerT,label="Energía Total")
ax1.legend(loc=0)

<IPython.core.display.Javascript object>